In [1]:
from subprocess import check_output
import subprocess
import threading
import os


def runServer(): 
    env = dict(os.environ)
    env['JAVA_OPTS'] = ''
    subprocess.call(['./startserver.bat'], env=env)


thread = threading.Thread(target=runServer, args=())
thread.start()

In [2]:
import py4j.java_gateway as jg
from py4j.java_gateway import JavaGateway

def run_model(model_string, steps, metric):
    gw = JavaGateway() # New gateway connection
    bridge = gw.entry_point
    bridge.openModel(model_string)
    setParamsRandom(gw,bridge, model_string)
    bridge.command("setup")
    bridge.command("repeat " + str(steps) + " [go]")
    return bridge.report("count turtles")

def setParamsRandom(gateway, bridge, model_string):
    paramSpecs = bridge.getParamList(model_string).getParamSpecs()

    for paramSpec in paramSpecs:
        if jg.is_instance_of(gateway,paramSpec,"bsearch.space.DoubleDiscreteSpec"):
            paramValue = paramSpec.generateRandomValue(gateway.jvm.org.nlogo.api.MersenneTwisterFast())
        if jg.is_instance_of(gateway,paramSpec,"bsearch.space.DoubleContinuousSpec"):
            paramValue = paramSpec.generateRandomValue(gateway.jvm.org.nlogo.api.MersenneTwisterFast())
        if jg.is_instance_of(gateway,paramSpec,"bsearch.space.CategoricalSpec"):
            paramValue = paramSpec.generateRandomValue(gateway.jvm.org.nlogo.api.MersenneTwisterFast())
        if jg.is_instance_of(gateway,paramSpec,"bsearch.space.ConstantSpec"):
            paramValue = paramSpec.generateRandomValue(gateway.jvm.org.nlogo.api.MersenneTwisterFast())
        print("set " + str(paramSpec.getParameterName()) + " " + str(paramValue))
        bridge.command("set " + str(paramSpec.getParameterName()) + " " + str(paramValue))
    


In [3]:
metric = "count households"
model_string = "C:/Program Files/NetLogo 6.0.2/app/models/Sample Models/Social Science/Unverified/Artificial Anasazi.nlogo"
print(run_model(model_string,  50, metric))

set map-view water sources
set harvest-adjustment 0.45
set harvest-variance 0.02
set death-age 30.0
set fertility-ends-age 33.0
set fertility 0.179
set historic-view? False
596.0
